In [12]:
!pip list | grep oak

oaklib                        0.5.8


In [13]:
from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A
adapter = get_adapter('pronto:hp-base.obo')
print(adapter.label("HP:0000001"))

All


In [ ]:
for hp in adapter.descendants("HP:0000001", predicates=[IS_A]):
    print(hp, adapter.label(hp))